# Домашнее задание по теме "Классификация: Логистическая регрессия и SVM"

- Имеются данные adult.csv (см. в материалах к занятию).
- Целевой переменной является уровень дохода income (крайний правый столбец).
- Описание признаков можно найти по ссылке www.cs.toronto.edu...etail.html
- Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
- Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.
- Готовый ноутбук выложить на гитхаб и прислать ссылку.

In [2]:
# импортируем библиотеку для работы с dataframe
import pandas as pd

In [4]:
# прочитаем файл csv
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,class
0,2,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States,<=50K
1,3,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States,<=50K
2,2,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States,<=50K
3,3,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States,<=50K
4,1,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba,<=50K


In [5]:
# выведем уникальные значения для столбца, который будем предсказывать
data['class'].unique()

array(['<=50K', '>50K'], dtype=object)

In [6]:
# посмотрим, что с данными, типы, пропуски, признаки
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      46033 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capitalgain     48842 non-null  int64 
 11  capitalloss     48842 non-null  int64 
 12  hoursperweek    48842 non-null  int64 
 13  native-country  47985 non-null  object
 14  class           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [7]:
# посмотрим на статистические параметры
data.describe()

,age,fnlwgt,education-num,capitalgain,capitalloss,hoursperweek
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,1.771078,1.896641e+05,10.078089,0.200319,0.114942,1.950698
std,1.295378,1.056040e+05,2.570973,0.746182,0.556924,0.897038
min,0.000000,1.228500e+04,1.000000,0.000000,0.000000,0.000000
25%,1.000000,1.175505e+05,9.000000,0.000000,0.000000,2.000000
50%,2.000000,1.781445e+05,10.000000,0.000000,0.000000,2.000000
75%,3.000000,2.376420e+05,12.000000,0.000000,0.000000,2.000000
max,4.000000,1.490400e+06,16.000000,4.000000,4.000000,4.000000


In [8]:
# выведем голову данных для столбца occupation, где нет данных
data[pd.isnull(data['occupation'])].head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,class
27,3,NaN,180211,Some-college,10,Married-civ-spouse,NaN,Husband,Asian-Pac-Islander,Male,0,0,3,South,>50K
61,1,NaN,293936,7th-8th,4,Married-spouse-absent,NaN,Not-in-family,White,Male,0,0,2,NaN,<=50K
69,0,NaN,200681,Some-college,10,Never-married,NaN,Own-child,White,Male,0,0,2,United-States,<=50K
77,4,NaN,212759,10th,6,Married-civ-spouse,NaN,Husband,White,Male,0,0,0,United-States,<=50K
106,0,NaN,304873,10th,6,Never-married,NaN,Own-child,White,Female,4,0,1,United-States,<=50K


In [9]:
#  занулим ячейки столбцов, где нет данных, чтобы работать с полными данными и получать качественный результат
data = data[pd.isnull(data['occupation']) == 0]
data = data[pd.isnull(data['workclass']) == 0]
data = data[pd.isnull(data['native-country']) == 0]

In [10]:
# поглядим на данные после обработки
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             45222 non-null  int64 
 1   workclass       45222 non-null  object
 2   fnlwgt          45222 non-null  int64 
 3   education       45222 non-null  object
 4   education-num   45222 non-null  int64 
 5   marital-status  45222 non-null  object
 6   occupation      45222 non-null  object
 7   relationship    45222 non-null  object
 8   race            45222 non-null  object
 9   sex             45222 non-null  object
 10  capitalgain     45222 non-null  int64 
 11  capitalloss     45222 non-null  int64 
 12  hoursperweek    45222 non-null  int64 
 13  native-country  45222 non-null  object
 14  class           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


## OneHotEncoder

In [13]:
# выберем категориальные признаки для использования в обучении 
selectedColumns = data [['race', 'education','native-country']]

In [14]:
# переведём наши выбранные категориальные признаки в индикаторные переменные  и выведем "голову"
X = pd.get_dummies(selectedColumns)
X.head()

,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### LabelEncoder

In [15]:
# теперь импортируем labelEncoder так же для преобразования нашей целевой категориальной переменной class
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [16]:
# натренируем методом fit модель
le.fit( data['class'])

LabelEncoder()

In [17]:
# посмотрим что внутри уникальные значения и тип
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [18]:
# посмотрим на процесс преобразования >50K = "1", а <=50K = "0"
le.transform( ['<=50K', '>50K', '<=50K']) # пример расшифровки

array([0, 1, 0])

In [19]:
# теперь наоборот из "чисел" в категории
le.inverse_transform([1, 0, 1])

array(['>50K', '<=50K', '>50K'], dtype=object)

In [20]:
# записываем в таргетную переменную "y" преобразованный столбец class
y = pd.Series (data = le.transform(data['class']))
y.tail()

45217    0
45218    0
45219    0
45220    0
45221    1
dtype: int32

## Логистическая регрессия

In [57]:
# импортируем логистическую регрессию из библиотеки sklearn для решения задачи
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline # используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
# make_pipeline как в конвейре сначала применяем стандартизацию для наших данных, а потом логистическую регрессию
model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

In [49]:
# а здесь пример без стандартизации (лучше со стандартизацией, а то что-то красное выскакивает=) )
model = LogisticRegression() # берем в качестве модели логистическую регресиию из scikit-learn

In [58]:
# разделяем наши данные на тренировочные для обучения(70%) и тестовые для проверки (30%) random_state закрепляет разделение, 
# чтобы не менялись случайным образом
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [59]:
# обучаем на части датасета (train)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [60]:
# посмотрим как выглядят наши предсказания
predictions[:5] 

array([1, 0, 0, 0, 0])

In [61]:
# получаем массив
model.predict(X_test) 

array([1, 0, 0, ..., 0, 0, 0])

In [62]:
# получаем массив вероятностых ожиданий
model.predict_proba(X_test)

array([[0.33940454, 0.66059546],
       [0.93660997, 0.06339003],
       [0.77734125, 0.22265875],
       ...,
       [0.85488078, 0.14511922],
       [0.77734125, 0.22265875],
       [0.77734125, 0.22265875]])

In [63]:
# оценка нашей модели на обучающих данных
model.score(X_train, y_train) 

0.7724214184173116

In [64]:
# оценка нашей модели на обобщающих(тестовых) данных
model.score(X_test,y_test)

0.7753372153018353

## Метод опорных векторов. SVM

In [66]:
from sklearn.svm import SVC
# используем пайплайны тот же конвейер со стандартизацией
clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [67]:
clf.score(X_train, y_train) 

0.7734639077554889

In [68]:
clf.score(X_test, y_test)

0.7740841748359991

In [69]:
# а тут поглядим на оценку нашей модели без стандартизации
svc = SVC(gamma='auto')
svc.fit(X_train, y_train)

SVC(gamma='auto')

In [74]:
svc.score(X_train, y_train)

0.7716948349391881

In [75]:
svc.score(X_test, y_test)

0.7747475492002653